# Avaliando os Embeddings

In [61]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    host="localhost", port="8000", settings=Settings(allow_reset=True)
)
collection = client.get_or_create_collection("olist_reviews")

Verificando a quantidade de registros

In [62]:
count = collection.count()
print(f"Quantidade de registros na coleção: {count}")

Quantidade de registros na coleção: 30342


Pegando alguns exemplos aleatórios

In [63]:
import random

all_ids = collection.get()["ids"]
random_ids = random.sample(all_ids, 1)

# Buscar os registros correspondentes
random_records = collection.get(ids=random_ids, include=["embeddings", "documents", "metadatas"])
random_records

{'ids': ['eedfdd1e4ef97bae3bc2a49f06e3016b_856b55eef75569bf5ec655e857bf84f7'],
 'embeddings': array([[ 0.07591171,  0.03293879,  0.0160067 , -0.01172744,  0.00581043,
          0.10624187,  0.3572781 , -0.03047876, -0.03980613, -0.16909057,
          0.1006026 , -0.02994548, -0.05006808,  0.18062666, -0.19405581,
          0.10068502,  0.00998913, -0.09508895, -0.072451  , -0.15743494,
         -0.21218112,  0.15791976, -0.1488847 ,  0.22834216, -0.23098463,
          0.02463485, -0.08491322,  0.03997763,  0.04840743, -0.2662214 ,
          0.08611382, -0.2750817 ,  0.06547759,  0.15898623, -0.01722305,
          0.04781552,  0.06412971, -0.00578718,  0.03604148, -0.0416218 ,
         -0.01665977, -0.00213949,  0.09425475, -0.05368577,  0.20396858,
          0.05234119,  0.18114004,  0.16142656,  0.38424173,  0.24229354,
         -0.13938582, -0.31167495, -0.1023287 ,  0.13089886, -0.01303229,
          0.1923083 , -0.03303931,  0.0107516 ,  0.05919843,  0.29067284,
          0.1940361

## Review Destaques (top reviews)

Para elencar os reviews em destaque vamos utilizar a busca por similaridade com um embedding médio dos reviews. Isso fará com que os reviews mais próximos do "centro" dos agrupamentos sejam selecionados.


Vamos calcular o embedding médio para o produto com id `a62e25e09e05e6faf31d90c6ec1aa3d1`

In [64]:
product_id = "a62e25e09e05e6faf31d90c6ec1aa3d1"
results = collection.get(
    where={"product_id": product_id},
    include=["embeddings", "metadatas", "documents"]
)

embeddings = results["embeddings"]
print(f"Total de embeddings: {len(embeddings)}")
print("Exemplos de Reviews:")
for i, (doc, metadata) in enumerate(zip(results["documents"], results["metadatas"])):
    print(f"Review {i + 1}: {doc} - Sentimento: {metadata['sentiment_label']}")
    if i >= 10:  # Limitar a exibição a 5 reviews
        break

Total de embeddings: 57
Exemplos de Reviews:
Review 1: Continue assim.o que tiver no site mantenha sempre - Sentimento: positivo
Review 2: So recebemos um relogio pelo correio

att - Sentimento: neutro
Review 3: Recomendo para todos, não tive problema nenhum, mercadoria e prazo ótimos. - Sentimento: positivo
Review 4: Sem problemas na execução do pedido, satisfação. - Sentimento: positivo
Review 5: OTIMO PRODUTO TUDO DENTRO DO ESPERADO - Sentimento: positivo
Review 6: Os relógios eram para serem iguais, vieram de marca e tamanhos diferentes - Sentimento: neutro
Review 7: Compro e recomendo - Sentimento: positivo
Review 8: produto chegou no prazo - Sentimento: positivo
Review 9: Amei o produto, chegou bem antes do prazo de entrega. - Sentimento: positivo
Review 10: bom - Sentimento: positivo
Review 11: Chegou muito rapido, excelente produto - Sentimento: positivo


In [65]:
import numpy as np

mean_embeddings = np.mean(embeddings, axis=0)
print(f"Dimensões do embedding médio: {mean_embeddings.shape}")

Dimensões do embedding médio: (384,)


In [73]:
results = collection.query(
    query_embeddings=[mean_embeddings],
    n_results=5,
    include=["metadatas", "documents"]
)

for i, (doc, metadata) in enumerate(zip(results["documents"][0], results["metadatas"][0])):
    print(f"Review {i + 1}: {doc} - Sentimento: {metadata['sentiment_label']}")

Review 1: entregaram antes da data, o produto é bom. 
obrigada - Sentimento: positivo
Review 2: Foi entregue no prazo, veio tudo correto, ganhei um mimos, amei meus produtos. Obrigada. Super recomendo.obg - Sentimento: positivo
Review 3: entregou antes do prazo.super recomendo - Sentimento: positivo
Review 4: chegou antes do prazo. Bom serviço - Sentimento: positivo
Review 5: Produto cfe o anunciado! Entregue no prazo... - Sentimento: positivo


> Como podemos ver os "Top Reviews" relacionados a esse produto são relacionados a entrega e todos classificados como positivos

In [72]:
from collections import Counter

reviews = collection.get(
    where={"product_id": product_id},
    include=["metadatas"]
)

sentiments = [meta["sentiment_label"] for meta in reviews["metadatas"]]
counts = Counter(sentiments)
total = sum(counts.values())

for label in ["positivo", "neutro", "negativo"]:
    percent = (counts.get(label, 0) / total) * 100
    print(f"{label.capitalize()}: {percent:.2f}%")


Positivo: 68.42%
Neutro: 10.53%
Negativo: 21.05%


> Mais da metade dos reviews desse produto exemplo que pegamos são positivos